# Setup

In [4]:
# Common imports
import os
import numpy as np
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
%matplotlib inline

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# Linear Regression

## Inverse Matrix using Tensorflow

In [14]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [15]:
print(theta_value)

[[ 2.06855845]
 [ 0.82961953]
 [ 0.11875179]
 [-0.26552707]
 [ 0.30569634]
 [-0.00450292]
 [-0.03932633]
 [-0.89988261]
 [-0.87053859]]


## Inverse Matrix using Numpy

In [17]:
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[ 2.06855817]
 [ 0.8296193 ]
 [ 0.11875165]
 [-0.26552688]
 [ 0.30569623]
 [-0.004503  ]
 [-0.03932627]
 [-0.89988565]
 [-0.870541  ]]


# Using Gradient Descent

## Manually Computed Gradient

$\nabla_\theta = \frac{\partial (X \theta - y)^T (X \theta - y)}{\partial \theta} = 2 X^T (X \theta - y)$

In [24]:
reset_graph()

n_epochs = 10000
learning_rate = 0.01

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 500 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.16154
Epoch 500 MSE = 0.543636
Epoch 1000 MSE = 0.530629
Epoch 1500 MSE = 0.526572
Epoch 2000 MSE = 0.52517
Epoch 2500 MSE = 0.524651
Epoch 3000 MSE = 0.524451
Epoch 3500 MSE = 0.524373
Epoch 4000 MSE = 0.524342
Epoch 4500 MSE = 0.524329
Epoch 5000 MSE = 0.524324
Epoch 5500 MSE = 0.524322
Epoch 6000 MSE = 0.524321
Epoch 6500 MSE = 0.524321
Epoch 7000 MSE = 0.524321
Epoch 7500 MSE = 0.524321
Epoch 8000 MSE = 0.524321
Epoch 8500 MSE = 0.524321
Epoch 9000 MSE = 0.524321
Epoch 9500 MSE = 0.524321
Best theta:
[[ 2.06855226]
 [ 0.82964426]
 [ 0.11875612]
 [-0.26557344]
 [ 0.30573449]
 [-0.00450158]
 [-0.03932724]
 [-0.89982933]
 [-0.87048846]]
